# Cleaning the raw data

## loading in Data and connecting to drive

In [ ]:
import os

# Mount the drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

# Bubs
# df = pd.read_excel('/content/drive/My Drive/Masters Thesis/Colab notebook/raw_data.xlsx')

# Sebs
df = pd.read_excel('/content/drive/MyDrive/Masters Thesis/Colab notebook/SPX500 data/raw_data.xlsx')

In [ ]:
# Check for missing Tweet Content
df['Tweet_Content'].isna().sum() # 291 rows

# Removing rows with nothing in tweet content
df_notweets = df.dropna(subset=['Tweet_Content'])

df_notweets.shape

(48363, 23)

In [ ]:
df_notweets

,Query_Str,Post_URL,Author_Name,Author_Web_Page_URL,Author_Handle,Verified_Status,UTC_Time,Ads,Tweet_Content,Post_ID,...,View_Count,Bookmark_Count,Tweet_Image_URL,Replying_to,Reply_to_Whom,Reply_to_Whom_URL,Reply_to_Whom_Username,Reply_to_Whom_Handle,Language,Follower_Count
12,(#spx500) min_replies:0 min_faves:0 min_retwee...,https://x.com/search?q=(#spx500) min_replies:0...,SanctionsAML,https://x.com/SanctionsAml,SanctionsAml,0.0,2022-01-01 11:04:50,False,2021 amid 10%+ #inflation (1980 methodology). ...,1.477234e+18,...,0,0,https://pbs.twimg.com/media/FIAw0K8X0AM7rdN.jp...,False,NaN,NaN,NaN,NaN,en,502
13,(#spx500) min_replies:0 min_faves:0 min_retwee...,https://x.com/search?q=(#spx500) min_replies:0...,SachinKsd1,https://x.com/sachinksd1,sachinksd1,1.0,2022-01-01 04:00:24,False,#DXY breaking down .. #SPX500 is on thin ICE ....,1.477128e+18,...,0,0,https://pbs.twimg.com/media/FH_Pq24VIAIc-dh.jpg,False,NaN,NaN,NaN,NaN,en,3302
14,(#spx500) min_replies:0 min_faves:0 min_retwee...,https://x.com/search?q=(#spx500) min_replies:0...,Raamraj Capital 🔰💰📈,https://x.com/Smartmoov2,Smartmoov2,0.0,2022-01-01 00:48:28,False,2021 📆 Asset Performance:\n\n#Gold ⛏ -4%\n#SPX...,1.477079e+18,...,0,0,https://pbs.twimg.com/media/FH-jvLHVgAQ9tIY.jpg,False,NaN,NaN,NaN,NaN,en,15
15,(#spx500) min_replies:0 min_faves:0 min_retwee...,https://x.com/search?q=(#spx500) min_replies:0...,Grizzly Bulls,https://x.com/GrizzlyBulls,GrizzlyBulls,0.0,2022-01-01 20:13:24,False,https://t.co/XDU4YXDujK\nWeekly Market Analysi...,1.477372e+18,...,0,0,https://pbs.twimg.com/media/FICuJZpXIAE8kXs.jpg,False,NaN,NaN,NaN,NaN,en,39
16,(#spx500) min_replies:0 min_faves:0 min_retwee...,https://x.com/search?q=(#spx500) min_replies:0...,Invariant Perspective,https://x.com/InvariantPersp1,InvariantPersp1,0.0,2022-01-01 04:22:59,False,#recession ... #StockMarket #Bubble edition\n\...,1.477133e+18,...,0,0,NaN,False,NaN,NaN,NaN,NaN,fr,5643
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48649,(#spx500) min_replies:0 min_faves:0 min_retwee...,https://x.com/search?q=(#spx500) min_replies:0...,Plus,https://x.com/plus_trades,plus_trades,1.0,2024-12-30 01:33:39,False,3️⃣0️⃣ 1️⃣2️⃣ 2️⃣0️⃣2️⃣4️⃣\n\n📌 #SPX500 -0.22%...,1.873543e+18,...,613,0,NaN,False,NaN,NaN,NaN,NaN,en,5824
48650,(#spx500) min_replies:0 min_faves:0 min_retwee...,https://x.com/search?q=(#spx500) min_replies:0...,LowCapCrypto,https://x.com/low_cap41279,low_cap41279,0.0,2024-12-30 01:36:22,False,@NYshitexchange $NYSE coming for #Msft #SPX500...,1.873544e+18,...,4,0,NaN,True,NYshitexchange,https://x.com/NYshitexchange,New York Shit Exchange $NYSE,NYshitexchange,en,17
48651,(#spx500) min_replies:0 min_faves:0 min_retwee...,https://x.com/search?q=(#spx500) min_replies:0...,Ghost_SnipeR 🐂💹💀,https://x.com/Crypto_Sniper__,Crypto_Sniper__,0.0,2024-12-30 14:17:50,False,Today will get s&amp;p dump and btc\nNot sure ...,1.873735e+18,...,57,0,NaN,False,NaN,NaN,NaN,NaN,en,36
48652,(#spx500) min_replies:0 min_faves:0 min_retwee...,https://x.com/search?q=(#spx500) min_replies:0...,Anuraag C,https://x.com/AhoNiranjana,AhoNiranjana,1.0,2024-12-30 17:29:07,False,🎯 Our Lunar Forecast Performance Review is LIV...,1.873783e+18,...,178,0,NaN,False,NaN,NaN,NaN,NaN,en,2559


In [ ]:
# checking for missing follower count
df_missing = df_notweets[df_notweets["Follower_Count"].isna()]

df_missing.shape #11151 follower count is missing

(145, 23)

There are a lot of follower counts that are missing. We will go through them manually to see why this is the case, for example perhaps the account has been deleted since scrapping or perhaps changed usernames. If this is the case we will decide whether or not the tweet content is important. However, to reduce a but of the manual work, we will first try scraping the usernames again to see if there was maybe a problem with the scraper.

In [ ]:
unique_users = df_missing['Author_Handle'].dropna().unique().tolist()

# Split the list into chunks of 100 items each
chunks = [unique_users[i:i + 100] for i in range(0, len(unique_users), 100)]

# Format each chunk as a string for easy readability
formatted_output = [f"[{', '.join(map(repr, chunk))}]" for chunk in chunks]

# Join the chunks into one final output
unique_users = ",\n\n".join(formatted_output)

# Print the formatted output
print(unique_users)


['be_livermoore', 'MoneyMa71574583', 'PhilipJagd', 'Fly_divyansh320', 'onchainfarhan', 'ReubenEmore', 'erc20_merlin', 'codexabcd', 'marslan1923', 'RoyalHappyMan', 'Bryptococcus', 'ContraryTrader', 'SupraFast7159', 'LocusTrading', 'H33xHive', 'noJuanbetter', 'sensitivesuit', 'Cr4zy_Chef', 'TTXAIAnalyst', 'PiusCrypFanatic', '1V1onlyy', 'shamiul293', 'cryptoblack48', 'cryptoblack46', 'alnasr_97_7', 'cryptoblack50', 'trading_RSI']


In [ ]:
df_missing.to_excel("/content/drive/My Drive/Masters Thesis/Colab notebook/missing_follower_count.xlsx", index=False)

running the code again, we end up with only 51 errors, reducing the manual workload by a lot. Now we will continue to clean the code so it is ready for sentiment analysis.

In [ ]:
# Read in the missing followers
df_new_followers = pd.read_excel('/content/drive/MyDrive/Masters Thesis/Colab notebook/X follower count data/final_scraper_results.xlsx')

# renaming column to match in both sheets
df_new_followers.rename(columns={"Username": "Author_Handle"}, inplace=True)

# make as a dictionary
follower_dict = df_new_followers.set_index("Author_Handle")["Follower Count"].to_dict()

# Update 'Follower_Count' column in df_notweets, keeping existing values where the Author_Handle is not found in follower_dict
df_notweets['Follower_Count'] = df_notweets['Follower_Count'].where(~df_notweets['Author_Handle'].isin(follower_dict.keys())) \
    .fillna(df_notweets['Author_Handle'].map(follower_dict))

<ipython-input-8-601d12b902ad>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_notweets['Follower_Count'] = df_notweets['Follower_Count'].where(~df_notweets['Author_Handle'].isin(follower_dict.keys())) \


In [ ]:
df_notweets['Follower_Count'].isna().sum() #231

np.int64(145)

manually checking, there are only 231 tweets that will be removed, as there are 51 users who's account have since been deleted. These will be removed from the dataset.

In [ ]:
df_notweets.shape

(48363, 23)

In [ ]:
df_notweets['Tweet_Content'] = df_notweets['Tweet_Content'].astype(str)  # Make sure Author_Handle is a string
df_notweets['Follower_Count'] = pd.to_numeric(df_notweets['Follower_Count'], errors='coerce')  # Ensure Follower_Count is numeri

df_notweets.dtypes


<ipython-input-11-bf2ba5cd98dc>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_notweets['Tweet_Content'] = df_notweets['Tweet_Content'].astype(str)  # Make sure Author_Handle is a string
<ipython-input-11-bf2ba5cd98dc>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_notweets['Follower_Count'] = pd.to_numeric(df_notweets['Follower_Count'], errors='coerce')  # Ensure Follower_Count is numeri


,0
Query_Str,object
Post_URL,object
Author_Name,object
Author_Web_Page_URL,object
Author_Handle,object
Verified_Status,float64
UTC_Time,datetime64[ns]
Ads,bool
Tweet_Content,object
Post_ID,float64


In [ ]:
df_notweets.shape

(48363, 23)

In [ ]:
df_notweets = df_notweets.dropna(subset=['Follower_Count'])
df_notweets.shape

(48216, 23)

In [ ]:
df_notweets

,Query_Str,Post_URL,Author_Name,Author_Web_Page_URL,Author_Handle,Verified_Status,UTC_Time,Ads,Tweet_Content,Post_ID,...,View_Count,Bookmark_Count,Tweet_Image_URL,Replying_to,Reply_to_Whom,Reply_to_Whom_URL,Reply_to_Whom_Username,Reply_to_Whom_Handle,Language,Follower_Count
12,(#spx500) min_replies:0 min_faves:0 min_retwee...,https://x.com/search?q=(#spx500) min_replies:0...,SanctionsAML,https://x.com/SanctionsAml,SanctionsAml,0.0,2022-01-01 11:04:50,False,2021 amid 10%+ #inflation (1980 methodology). ...,1.477234e+18,...,0,0,https://pbs.twimg.com/media/FIAw0K8X0AM7rdN.jp...,False,NaN,NaN,NaN,NaN,en,502.0
13,(#spx500) min_replies:0 min_faves:0 min_retwee...,https://x.com/search?q=(#spx500) min_replies:0...,SachinKsd1,https://x.com/sachinksd1,sachinksd1,1.0,2022-01-01 04:00:24,False,#DXY breaking down .. #SPX500 is on thin ICE ....,1.477128e+18,...,0,0,https://pbs.twimg.com/media/FH_Pq24VIAIc-dh.jpg,False,NaN,NaN,NaN,NaN,en,3302.0
14,(#spx500) min_replies:0 min_faves:0 min_retwee...,https://x.com/search?q=(#spx500) min_replies:0...,Raamraj Capital 🔰💰📈,https://x.com/Smartmoov2,Smartmoov2,0.0,2022-01-01 00:48:28,False,2021 📆 Asset Performance:\n\n#Gold ⛏ -4%\n#SPX...,1.477079e+18,...,0,0,https://pbs.twimg.com/media/FH-jvLHVgAQ9tIY.jpg,False,NaN,NaN,NaN,NaN,en,15.0
15,(#spx500) min_replies:0 min_faves:0 min_retwee...,https://x.com/search?q=(#spx500) min_replies:0...,Grizzly Bulls,https://x.com/GrizzlyBulls,GrizzlyBulls,0.0,2022-01-01 20:13:24,False,https://t.co/XDU4YXDujK\nWeekly Market Analysi...,1.477372e+18,...,0,0,https://pbs.twimg.com/media/FICuJZpXIAE8kXs.jpg,False,NaN,NaN,NaN,NaN,en,39.0
16,(#spx500) min_replies:0 min_faves:0 min_retwee...,https://x.com/search?q=(#spx500) min_replies:0...,Invariant Perspective,https://x.com/InvariantPersp1,InvariantPersp1,0.0,2022-01-01 04:22:59,False,#recession ... #StockMarket #Bubble edition\n\...,1.477133e+18,...,0,0,NaN,False,NaN,NaN,NaN,NaN,fr,5643.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48649,(#spx500) min_replies:0 min_faves:0 min_retwee...,https://x.com/search?q=(#spx500) min_replies:0...,Plus,https://x.com/plus_trades,plus_trades,1.0,2024-12-30 01:33:39,False,3️⃣0️⃣ 1️⃣2️⃣ 2️⃣0️⃣2️⃣4️⃣\n\n📌 #SPX500 -0.22%...,1.873543e+18,...,613,0,NaN,False,NaN,NaN,NaN,NaN,en,5824.0
48650,(#spx500) min_replies:0 min_faves:0 min_retwee...,https://x.com/search?q=(#spx500) min_replies:0...,LowCapCrypto,https://x.com/low_cap41279,low_cap41279,0.0,2024-12-30 01:36:22,False,@NYshitexchange $NYSE coming for #Msft #SPX500...,1.873544e+18,...,4,0,NaN,True,NYshitexchange,https://x.com/NYshitexchange,New York Shit Exchange $NYSE,NYshitexchange,en,17.0
48651,(#spx500) min_replies:0 min_faves:0 min_retwee...,https://x.com/search?q=(#spx500) min_replies:0...,Ghost_SnipeR 🐂💹💀,https://x.com/Crypto_Sniper__,Crypto_Sniper__,0.0,2024-12-30 14:17:50,False,Today will get s&amp;p dump and btc\nNot sure ...,1.873735e+18,...,57,0,NaN,False,NaN,NaN,NaN,NaN,en,36.0
48652,(#spx500) min_replies:0 min_faves:0 min_retwee...,https://x.com/search?q=(#spx500) min_replies:0...,Anuraag C,https://x.com/AhoNiranjana,AhoNiranjana,1.0,2024-12-30 17:29:07,False,🎯 Our Lunar Forecast Performance Review is LIV...,1.873783e+18,...,178,0,NaN,False,NaN,NaN,NaN,NaN,en,2559.0


In [ ]:
# rename tweets to xpost for more accuracy
df_notweets.rename(columns={'Tweet_Content': 'X_Post'}, inplace=True)
df_notweets.rename(columns={'UTC_Time': 'Date'}, inplace=True)

# select certain column
final_data = df_notweets[['Author_Handle', 'Date','X_Post', 'Reply_Count', 'Repost_Count', 'Like_Count', 'View_Count', 'Follower_Count', 'Replying_to', 'Verified_Status']]
final_data

<ipython-input-15-3d611bd5fceb>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_notweets.rename(columns={'Tweet_Content': 'X_Post'}, inplace=True)
<ipython-input-15-3d611bd5fceb>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_notweets.rename(columns={'UTC_Time': 'Date'}, inplace=True)


,Author_Handle,Date,X_Post,Reply_Count,Repost_Count,Like_Count,View_Count,Follower_Count,Replying_to,Verified_Status
12,SanctionsAml,2022-01-01 11:04:50,2021 amid 10%+ #inflation (1980 methodology). ...,0.0,0,0,0,502.0,False,0.0
13,sachinksd1,2022-01-01 04:00:24,#DXY breaking down .. #SPX500 is on thin ICE ....,1.0,1,0,0,3302.0,False,1.0
14,Smartmoov2,2022-01-01 00:48:28,2021 📆 Asset Performance:\n\n#Gold ⛏ -4%\n#SPX...,0.0,0,0,0,15.0,False,0.0
15,GrizzlyBulls,2022-01-01 20:13:24,https://t.co/XDU4YXDujK\nWeekly Market Analysi...,0.0,0,0,0,39.0,False,0.0
16,InvariantPersp1,2022-01-01 04:22:59,#recession ... #StockMarket #Bubble edition\n\...,0.0,2,0,0,5643.0,False,0.0
...,...,...,...,...,...,...,...,...,...,...
48649,plus_trades,2024-12-30 01:33:39,3️⃣0️⃣ 1️⃣2️⃣ 2️⃣0️⃣2️⃣4️⃣\n\n📌 #SPX500 -0.22%...,0.0,1,4,613,5824.0,False,1.0
48650,low_cap41279,2024-12-30 01:36:22,@NYshitexchange $NYSE coming for #Msft #SPX500...,0.0,0,0,4,17.0,True,0.0
48651,Crypto_Sniper__,2024-12-30 14:17:50,Today will get s&amp;p dump and btc\nNot sure ...,0.0,0,0,57,36.0,False,0.0
48652,AhoNiranjana,2024-12-30 17:29:07,🎯 Our Lunar Forecast Performance Review is LIV...,0.0,0,0,178,2559.0,False,1.0


In [ ]:
# Check for duplicates
duplicates = final_data[final_data.duplicated()] # 76 duplicates

# Remove duplicates
final_data = final_data.drop_duplicates()

# Check if duplicates are removed
print(f"Shape after removing duplicates: {final_data.shape}")


Shape after removing duplicates: (48140, 10)


In [ ]:
# Remove links from 'X_Post' column
final_data['X_Post'] = final_data['X_Post'].str.replace(r'http\S+', '', regex=True)

<ipython-input-17-ce995480a51c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_data['X_Post'] = final_data['X_Post'].str.replace(r'http\S+', '', regex=True)


In [ ]:
final_data.to_excel("/content/drive/MyDrive/Masters Thesis/Colab notebook/SPX500 data/final_SPX500_data1.xlsx", index=False)

## Matching and adding columns
Loading in the RoBERTa_sentiment_preds and final_SPX500_data1 datasets and merging 'Replying_to' column with final_data, to add the 'Replying_to' column to the RoBERTa_sentiment_preds dataset.

In [ ]:
import pandas as pd
# Loading in the RoBERTa predictions dataset
roberta_data = pd.read_excel('/content/drive/MyDrive/Masters Thesis/Colab notebook/Sentiment analysis/RoBERTa_sentiment_preds.xlsx')

# Loading in the final dataset before predictions
final_data = pd.read_excel('/content/drive/MyDrive/Masters Thesis/Colab notebook/SPX500 data/final_SPX500_data1.xlsx')

In [ ]:
# Checking shape of datasets
print(f"Shape of RoBERTa dataset: {roberta_data.shape}")
print(f"Shape of final dataset: {final_data.shape}")

Shape of RoBERTa dataset: (48140, 11)
Shape of final dataset: (48140, 10)


In [ ]:
# Create a copy of the original roberta_data
roberta_data_reply = roberta_data.copy()

# Ensure each X_Post has a unique Replying_to by dropping duplicates.
final_data_unique = final_data.drop_duplicates(subset="X_Post")

# Create a mapping Series where the index is X_Post and the value is Replying_to.
reply_mapping = final_data_unique.set_index("X_Post")["Replying_to"]

# Add the Replying_to column to roberta_data by mapping the X_Post values.
roberta_data_reply["Replying_to"] = roberta_data_reply["X_Post"].map(reply_mapping)


Checking to see if dataframe changed

In [ ]:
print(f"Shape after removing duplicates: {roberta_data_reply.shape}")

Shape after removing duplicates: (48140, 12)


Saving the RoBERTa dataset with the 'Replying_to' column as an excel file.

In [ ]:
roberta_data_reply.to_excel("/content/drive/MyDrive/Masters Thesis/Colab notebook/Sentiment analysis/RoBERTa_sentiment_preds_reply.xlsx", index=False)